# Data Cleaning

Name: Ningxin Kang (nik010@ucsd.edu)

Last update: 10/04/2022

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xlrd
import re

## 1. Formatting and cleaning the overall dataset

In [2]:
# Import the data
df_files_BHB = pd.read_excel('/Users/sylvia618/Desktop/Chen Lab/hek cells/input/BHBDiet_allfiles/filepath_file_BHB.xls')
df_files_CTRL = pd.read_excel('/Users/sylvia618/Desktop/Chen Lab/hek cells/input/BHBDiet_allfiles/filepath_file_CTRL.xls')

In [5]:
# Create two new dataframe for storing the data
df_BHB = pd.DataFrame()
df_CTRL = pd.DataFrame()

# Read file and concate by line for BHB group
tmp = [df_BHB]
for file in df_files_BHB['File path']:
    tmp.append(pd.read_excel(file))
df_BHB = pd.concat(tmp)
df_BHB = df_BHB.reset_index(drop = True)
print(df_BHB.head())

# Read file and concate by line for CTRL group
tmp = [df_CTRL]
for file in df_files_CTRL['File path']:
    tmp.append(pd.read_excel(file))
df_CTRL = pd.concat(tmp)
df_CTRL = df_CTRL.reset_index(drop = True)
print(df_CTRL.head())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/danielmunozmayorga/Documents/BHB_HEK293T_TauCell_allimages/Analyzed_images/Count Results/results_LG355_204_GFP/Outlines_results_LG355_204_GFP_1.xls'

In [ ]:
# Merge the two dataframe with new column categorize rows with diet group
df_BHB["category"] = "BHB"
df_CTRL["category"] = "CTRL"
df_sum = pd.merge(df_BHB,df_CTRL,how = "outer")

# Filter the aggregates by area
df_sum = df_sum[df_sum.Area > 0.4160e-09]
df_sum = df_sum[df_sum.Area < 1.5e-7]

# change colname
df_sum.columns.values[0] = "sample_index"

# temp line
#df_sum.to_csv("filtered_agg+CTRL.csv",sep = ",",index = False)

                                                      Label          Area  \
0      1  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  1.414000e-07   
1      2  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  1.664000e-08   
2      3  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  8.320000e-09   
3      4  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  6.656000e-08   
5      6  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  9.569000e-08   
...   ..                                                ...           ...   
5653  68  LG875_47_6.5_CTRL_DAPI100MS_GFP100MS_10X_4_XY1...  8.320000e-09   
5654  69  LG875_47_6.5_CTRL_DAPI100MS_GFP100MS_10X_4_XY1...  4.160000e-09   
5655  70  LG875_47_6.5_CTRL_DAPI100MS_GFP100MS_10X_4_XY1...  4.160000e-09   
5656  71  LG875_47_6.5_CTRL_DAPI100MS_GFP100MS_10X_4_XY1...  4.160000e-09   
5657  72  LG875_47_6.5_CTRL_DAPI100MS_GFP100MS_10X_4_XY1...  4.160000e-09   

       Mean  Mode  Min  Max    IntDen  RawIntDen category  
0     255.0   2

C:\Users\jamfo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\merge.py:1219: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(


In [3]:
#tmp line
df_sum = pd.read_csv('/Users/sylvia618/Desktop/Chen Lab/hek cells/input/filtered_agg+CTRL.csv',sep = ',')
# Scale the IntDen from square cetimeter to square micrometer
df_sum["Area"] = [area*10e8 for area in df_sum["Area"]]

# Extract the id of mouse from column 'Label'
df_sum['mouse_id'] = [re.search('LG.*_.*_.*[BL]',df_sum['Label'][i]).group() for i in range(df_sum.shape[0])]
# Extract the if of picture from column 'Label'
df_sum['picture_id'] = [re.search('^((?:[^_]*\_){8})([^_]*)',df_sum['Label'][i]).group(2)
                        for i in range(df_sum.shape[0])]
df_sum['picture_id'] = df_sum['mouse_id'] + '_' + df_sum['picture_id']      # combine mouth id and picture id

# check the updated dataframe
print(df_sum.head())
# Output formated file
df_sum.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/filtered_agg+CTRL_1.csv",sep = ",",index = False)

   sample_index                                              Label    Area  \
0             1  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...  141.40   
1             2  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...   16.64   
2             3  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...    8.32   
3             4  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...   66.56   
4             6  LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...   95.69   

    Mean  Mode  Min  Max    IntDen  RawIntDen category       mouse_id  \
0  255.0   255  255  255  0.000036       8670      BHB  LG355_204_BHB   
1  255.0   255  255  255  0.000004       1020      BHB  LG355_204_BHB   
2  255.0   255  255  255  0.000002        510      BHB  LG355_204_BHB   
3  255.0   255  255  255  0.000017       4080      BHB  LG355_204_BHB   
4  255.0   255  255  255  0.000024       5865      BHB  LG355_204_BHB   

        picture_id  
0  LG355_204_BHB_1  
1  LG355_204_BHB_1  
2  LG355_204_BHB_1  
3  LG355

## 2. Create file for number of aggregates per picture and per mouse

In [4]:
# Count the number of aggregates per plate
avg_agg_per_pic = pd.DataFrame(df_sum.groupby(['picture_id']).count())
# Format the dataframe by change rownames to column and use index as rownames
avg_agg_per_pic = avg_agg_per_pic.rename_axis("picture_id").reset_index()
# extract mouse id
avg_agg_per_pic['mouse_id'] = [re.search('LG.*_.*_.*[BL]',avg_agg_per_pic['picture_id'][i]).group()
                        for i in range(avg_agg_per_pic.shape[0])]
# extract category
avg_agg_per_pic['category'] = [avg_agg_per_pic['mouse_id'][i].split('_')[-1]
                        for i in range(avg_agg_per_pic.shape[0])]
# Rename and extract the targeted column
avg_agg_per_pic.rename({'sample_index': 'num_aggregates'}, axis=1, inplace=True)
avg_agg_per_pic = avg_agg_per_pic.iloc[:,[-1,-2,0,1]]

# Check dataframe
print(avg_agg_per_pic.head())

# Output dataframe
avg_agg_per_pic.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/avg_agg_per_pic.csv",sep = ",",index = False)

         mouse_id category        picture_id  num_aggregates
0   LG355_204_BHB      BHB   LG355_204_BHB_1             127
1   LG355_204_BHB      BHB   LG355_204_BHB_2              84
2   LG355_204_BHB      BHB   LG355_204_BHB_3              86
3   LG355_204_BHB      BHB   LG355_204_BHB_4              93
4  LG355_206_CTRL     CTRL  LG355_206_CTRL_1             178


In [5]:
# Calculate mean count for each mouse
avg_agg_per_mouse = pd.DataFrame(avg_agg_per_pic.groupby(['mouse_id']).mean())
# Format the dataframe by change rownames to column and use index as rownames
avg_agg_per_mouse = avg_agg_per_mouse.rename_axis("mouse_id").reset_index()
# extract category
avg_agg_per_mouse['category'] = [avg_agg_per_mouse['mouse_id'][i].split('_')[-1]
                        for i in range(avg_agg_per_mouse.shape[0])]
# check dataframe
print(avg_agg_per_mouse)

# Output dataframe
avg_agg_per_mouse.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/avg_agg_per_mouse.csv",sep = ",",index = False)

             mouse_id  num_aggregates category
0       LG355_204_BHB           97.50      BHB
1      LG355_206_CTRL          222.25     CTRL
2      LG355_207_CTRL          152.75     CTRL
3  LG355_208_6.5_CTRL          121.25     CTRL
4   LG355_210_6.5_BHB           99.75      BHB
5        LG875_43_BHB           57.00      BHB
6        LG875_44_BHB          164.00      BHB
7   LG875_46_6.5_CTRL          191.50     CTRL
8   LG875_47_6.5_CTRL           49.00     CTRL


## 3. Create file for average area and integrated density per picture and per mouse for single aggregates

In [59]:
# Calculate mean count for each mouse
avg_area_per_pic = pd.DataFrame(df_sum.groupby(['picture_id']).mean())
# Format the dataframe by change rownames to column and use index as rownames
avg_area_per_pic = avg_area_per_pic.rename_axis("picture_id").reset_index()
# extract mouse id
avg_area_per_pic['mouse_id'] = [re.search('LG.*_.*_.*[BL]',avg_area_per_pic['picture_id'][i]).group()
                        for i in range(avg_area_per_pic.shape[0])]
# extract category
avg_area_per_pic['category'] = [avg_area_per_pic['mouse_id'][i].split('_')[-1]
                        for i in range(avg_area_per_pic.shape[0])]
avg_area_per_pic = avg_area_per_pic.iloc[:,[-1,-2,0,2,7]]

print(avg_area_per_pic.head())
# Output dataframe
avg_area_per_pic.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/avg_area&IntDen_per_pic.csv",sep = ",",index = False)

  category        mouse_id        picture_id       Area    IntDen
0      BHB   LG355_204_BHB   LG355_204_BHB_1  38.685354  0.000010
1      BHB   LG355_204_BHB   LG355_204_BHB_2  48.586905  0.000012
2      BHB   LG355_204_BHB   LG355_204_BHB_3  47.165814  0.000012
3      BHB   LG355_204_BHB   LG355_204_BHB_4  44.865376  0.000011
4     CTRL  LG355_206_CTRL  LG355_206_CTRL_1  26.736573  0.000007


In [61]:
# Calculate mean count for each mouse
avg_area_per_mouse = pd.DataFrame(avg_area_per_pic.groupby(['mouse_id']).mean())
# Format the dataframe by change rownames to column and use index as rownames
avg_area_per_mouse = avg_area_per_mouse.rename_axis("mouse_id").reset_index()
# extract category
avg_area_per_mouse['category'] = [avg_area_per_mouse['mouse_id'][i].split('_')[-1]
                        for i in range(avg_area_per_mouse.shape[0])]

print(avg_area_per_mouse.head())
# Output dataframe
avg_area_per_mouse.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/avg_area&IntDen_per_mouse.csv",sep = ",",index = False)

             mouse_id       Area    IntDen category
0       LG355_204_BHB  44.825862  0.000011      BHB
1      LG355_206_CTRL  23.497216  0.000006     CTRL
2      LG355_207_CTRL  21.842932  0.000006     CTRL
3  LG355_208_6.5_CTRL  31.804666  0.000008     CTRL
4   LG355_210_6.5_BHB  43.624821  0.000011      BHB


## 4. Create file for total area and integrated density on a single well per picture and per mouse

In [76]:
# Calculate mean count for each mouse
total_area_per_pic = pd.DataFrame(df_sum.groupby(['picture_id']).sum())
# Format the dataframe by change rownames to column and use index as rownames
total_area_per_pic = total_area_per_pic.rename_axis("picture_id").reset_index()
# extract mouse id
total_area_per_pic['mouse_id'] = [re.search('LG.*_.*_.*[BL]',total_area_per_pic['picture_id'][i]).group()
                        for i in range(total_area_per_pic.shape[0])]
# extract category
total_area_per_pic['category'] = [total_area_per_pic['mouse_id'][i].split('_')[-1]
                        for i in range(total_area_per_pic.shape[0])]
total_area_per_pic = total_area_per_pic.iloc[:,[-1,-2,0,2,7]]

print(total_area_per_pic.head())
# Output dataframe
total_area_per_pic.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/total_area&IntDen_per_pic.csv",sep = ",",index = False)

  category        mouse_id        picture_id     Area    IntDen
0      BHB   LG355_204_BHB   LG355_204_BHB_1  4913.04  0.001253
1      BHB   LG355_204_BHB   LG355_204_BHB_2  4081.30  0.001041
2      BHB   LG355_204_BHB   LG355_204_BHB_3  4056.26  0.001034
3      BHB   LG355_204_BHB   LG355_204_BHB_4  4172.48  0.001064
4     CTRL  LG355_206_CTRL  LG355_206_CTRL_1  4759.11  0.001214


In [77]:
# Calculate mean count for each mouse
total_area_per_mouse = pd.DataFrame(total_area_per_pic.groupby(['mouse_id']).mean())
# Format the dataframe by change rownames to column and use index as rownames
total_area_per_mouse = total_area_per_mouse.rename_axis("mouse_id").reset_index()
# extract category
total_area_per_mouse['category'] = [total_area_per_mouse['mouse_id'][i].split('_')[-1]
                        for i in range(total_area_per_mouse.shape[0])]

print(total_area_per_mouse.head())
# Output dataframe
total_area_per_mouse.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/total_area&IntDen_per_mouse.csv",sep = ",",index = False)

             mouse_id       Area    IntDen category
0       LG355_204_BHB  4305.7700  0.001098      BHB
1      LG355_206_CTRL  4836.1000  0.001233     CTRL
2      LG355_207_CTRL  3327.0475  0.000848     CTRL
3  LG355_208_6.5_CTRL  3086.7700  0.000790     CTRL
4   LG355_210_6.5_BHB  4420.1325  0.001127      BHB


## 5. Input DAPI count, calculate `total area/DAPI` and `total IntDen/DAPI`

In [89]:
# Import the data
df_file_DAPI = pd.read_excel('/Users/sylvia618/Desktop/Chen Lab/hek cells/input/DAPI/filepath.xls')
# Create two new dataframe for storing the data
df_DAPI = pd.DataFrame(columns=['Slice', 'Count', 'Total Area', 'Average Size', 'Area Fraction',
       'Mean'])

# Read file and concate by line for BHB group
for file in df_file_DAPI['Filepath']:
    df_DAPI = pd.merge(df_DAPI, pd.read_csv((file), sep = "\t"), how = "outer")

df_DAPI.rename({'Slice': 'Label'}, axis=1, inplace=True)
# unify format
df_DAPI['Label'] = [label.replace("GFP100MS", "GFP_100ms") for label in df_DAPI['Label']]
# Extract the id of mouse from column 'Label'
df_DAPI['mouse_id'] = [re.search('LG.*_.*_.*[BL]',df_DAPI['Label'][i]).group() for i in range(df_DAPI.shape[0])]
# Extract the if of picture from column 'Label'
df_DAPI['picture_id'] = [re.search('^((?:[^_]*\_){8})([^_]*)',df_DAPI['Label'][i]).group(2)
                        for i in range(df_DAPI.shape[0])]
df_DAPI['picture_id'] = df_DAPI['mouse_id'] + '_' + df_DAPI['picture_id']      # combine mouth id and picture id

print(df_DAPI)

                                                Label  Count  Total Area  \
0   LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_1_XY...   2472    0.002000   
1   LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_2_XY...   2383    0.002000   
2   LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_3_XY...   2453    0.002000   
3   LG355_204_BHB_6.5_10x_DAPI100ms_GFP_100ms_4_XY...   2527    0.002000   
4   LG355_206_CTRL_6.5_10x_DAPI100ms_GFP_100ms_3_X...    235    0.000064   
5   LG355_206_CTRL_6.5_10x_DAPI100ms_GFP_100ms_2_X...   1984    0.001000   
6   LG355_206_CTRL_6.5_10x_DAPI100ms_GFP_100ms_3_X...   2333    0.002000   
7   LG355_206_CTRL_6.5_10x_DAPI100ms_GFP_100ms_4_X...   2307    0.002000   
8   LG355_207_CTRL_6.5_10x_DAPI100ms_GFP_100ms_1_X...   2475    0.002000   
9   LG355_207_CTRL_6.5_10x_DAPI100ms_GFP_100ms_2_X...   2279    0.002000   
10  LG355_207_CTRL_6.5_10x_DAPI100ms_GFP_100ms_3_X...   1961    0.002000   
11  LG355_207_CTRL_6.5_10x_DAPI100ms_GFP_100ms_4_X...   2403    0.002000   
12  LG355_20

In [97]:
# merge the total area of aggregates per pic and DAPI by picture id
area_vs_DAPI_per_pic = pd.merge(total_area_per_pic, df_DAPI, how = "inner", on = "picture_id")
# Calculate total area per DAPI and total IntDen per DAPI
area_vs_DAPI_per_pic['area_vs_DAPI'] = area_vs_DAPI_per_pic['Area']/area_vs_DAPI_per_pic['Count']
area_vs_DAPI_per_pic['IntDen_vs_DAPI']= area_vs_DAPI_per_pic['IntDen']/area_vs_DAPI_per_pic['Count']
# Select columns
area_vs_DAPI_per_pic = area_vs_DAPI_per_pic.iloc[:,[0,1,2,3,4,6,12,13]]
# rename column
area_vs_DAPI_per_pic.rename({'mouse_id_x': 'mouse_id'}, axis=1, inplace=True)
# Check output
print(area_vs_DAPI_per_pic.head())
# Output dataframe
area_vs_DAPI_per_pic.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/area&IntDen_vs_DAPI_per_pic.csv",sep = ",",index = False)

  category        mouse_id        picture_id     Area    IntDen  Count  \
0      BHB   LG355_204_BHB   LG355_204_BHB_1  4913.04  0.001253   2472   
1      BHB   LG355_204_BHB   LG355_204_BHB_2  4081.30  0.001041   2383   
2      BHB   LG355_204_BHB   LG355_204_BHB_3  4056.26  0.001034   2453   
3      BHB   LG355_204_BHB   LG355_204_BHB_4  4172.48  0.001064   2527   
4     CTRL  LG355_206_CTRL  LG355_206_CTRL_2  3893.81  0.000993   1984   

   area_vs_DAPI  IntDen_vs_DAPI  
0      1.987476    5.068390e-07  
1      1.712673    4.367302e-07  
2      1.653592    4.216641e-07  
3      1.651159    4.210819e-07  
4      1.962606    5.004884e-07  


In [98]:
# Calculate mean count for each mouse
area_vs_DAPI_per_mouse = pd.DataFrame(area_vs_DAPI_per_pic.groupby(['mouse_id']).mean())
# Format the dataframe by change rownames to column and use index as rownames
area_vs_DAPI_per_mouse = area_vs_DAPI_per_mouse.rename_axis("mouse_id").reset_index()
# extract category
area_vs_DAPI_per_mouse['category'] = [area_vs_DAPI_per_mouse['mouse_id'][i].split('_')[-1]
                        for i in range(area_vs_DAPI_per_mouse.shape[0])]

print(area_vs_DAPI_per_mouse.head())
# Output dataframe
area_vs_DAPI_per_mouse.to_csv("/Users/sylvia618/Desktop/Chen Lab/hek cells/input/area&IntDen_vs_DAPI_per_mouse.csv",sep = ",",index = False)

             mouse_id       Area    IntDen    Count  area_vs_DAPI  \
0       LG355_204_BHB  4305.7700  0.001098  2458.75      1.751225   
1      LG355_206_CTRL  5132.5450  0.001309  1714.75      7.966414   
2      LG355_207_CTRL  3327.0475  0.000848  2279.50      1.491791   
3  LG355_208_6.5_CTRL  3086.7700  0.000790  2486.50      1.264507   
4   LG355_210_6.5_BHB  4420.1325  0.001127  2359.00      1.920180   

   IntDen_vs_DAPI category  
0    4.465788e-07      BHB  
1    2.031499e-06     CTRL  
2    3.804247e-07     CTRL  
3    3.235784e-07     CTRL  
4    4.896706e-07      BHB  
